In [2]:
! pip install shioaji

  Using cached shioaji-0.1.2.dev9-cp37-cp37m-manylinux1_x86_64.whl (4.4 MB)
  Using cached requests-2.22.0-py2.py3-none-any.whl (57 kB)
  Using cached xxhash-1.4.3-cp37-cp37m-manylinux2010_x86_64.whl (234 kB)
  Using cached pydantic-1.0-cp37-cp37m-manylinux1_x86_64.whl (5.7 MB)
  Using cached sentry_sdk-0.14.1-py2.py3-none-any.whl (93 kB)
  Using cached pysolace-0.2.5-cp37-cp37m-manylinux2010_x86_64.whl (5.9 MB)
  Using cached loguru-0.4.1-py3-none-any.whl (54 kB)
  Using cached idna-2.8-py2.py3-none-any.whl (58 kB)
     |████████████████████████████████| 275 kB 1.1 MB/s eta 0:00:01
  Attempting uninstall: idna
    Found existing installation: idna 2.9
    Uninstalling idna-2.9:
      Successfully uninstalled idna-2.9
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [1]:
import json
import datetime
import shioaji as sj
import dolphindb
import pandas as pd
import yvis

In [2]:
# dsess = dolphindb.session()
# dsess.connect("localhost", 8848, 'admin', 'sts.ec6.sts.130')

In [3]:
api = sj.Shioaji()

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [4]:
with open('login.json', 'r') as f:
    login_kwargs = json.loads(f.read())

In [5]:
api.login(**login_kwargs, hashed=True, contracts_cb=print)

[FutureAccount(person_id='E224477313', broker_id='F002000', account_id='1481679', signed=True, username='蕭采容'),
 Account(account_type=<AccountType.H: 'H'>, person_id='E224477313', broker_id='9A95', account_id='0082039', signed=True, username='蕭采容'),
 StockAccount(person_id='E224477313', broker_id='9A95', account_id='0518472', signed=True, username='蕭采容\u3000\u3000')]

SecurityType.Index
SecurityType.Future
SecurityType.Option
SecurityType.Stock


In [6]:
c = api.Contracts.Stocks['2313']
c

Stock(exchange=<Exchange.TSE: 'TSE'>, code='2313', symbol='TSE2313', name='華通', category='28', limit_up=32.3, limit_down=26.5, reference=29.4, update_date='2020/03/26', day_trade=<DayTrade.Yes: 'Yes'>)

In [39]:
ticks_data = api.ticks(c, date=datetime.date.today().strftime("%Y-%m-%d"), timeout=100000)

In [40]:
df_ticks = pd.DataFrame({**ticks_data})

In [41]:
df_ticks['ts'] = pd.to_datetime(df_ticks['ts'])

In [10]:
df_ticks.iloc[-1]

bid_price                   31.25
ask_volume                    152
volume                        189
ts            2020-03-26 14:30:00
ask_price                    31.3
close                        31.3
bid_volume                     30
Name: 12522, dtype: object

In [11]:
df_ticks_init = df_ticks.head(10)

In [12]:
df_ticks_init[['ts', 'close', 'volume', 'ask_price', 'ask_volume', 'bid_price', 'bid_volume']]

,ts,close,volume,ask_price,ask_volume,bid_price,bid_volume
0,2020-03-26 09:00:05.064664,30.10,1537,30.1,21,30.05,104
1,2020-03-26 09:00:05.663441,30.05,2,30.1,32,30.05,102
2,2020-03-26 09:00:06.250607,30.05,2,30.1,33,30.05,100
3,2020-03-26 09:00:06.297073,30.05,2,30.1,33,30.05,98
4,2020-03-26 09:00:06.305245,30.05,1,30.1,33,30.05,97
5,2020-03-26 09:00:06.603459,30.10,3,30.1,30,30.05,97
6,2020-03-26 09:00:07.172143,30.05,1,30.1,30,30.05,96
7,2020-03-26 09:00:07.214110,30.05,6,30.1,30,30.05,90
8,2020-03-26 09:00:07.215635,30.05,2,30.1,30,30.05,88
9,2020-03-26 09:00:08.145023,30.10,1,30.1,29,30.05,89


In [13]:
c.reference

29.4

In [18]:
chart = yvis.PVChart(date=[], price=[], volume=[], 
                     price_max=c.limit_up, price_min=c.limit_down, 
                     price_ref=c.reference)

In [53]:
chart = yvis.PVChart(date=df_ticks_init.ts.astype(str).map(lambda x: x[:-3]).tolist(), 
                     price=df_ticks_init.close.tolist(), 
                     volume=df_ticks_init.volume.tolist(), 
                     price_max=c.limit_up, price_min=c.limit_down, price_ref=c.reference)

In [ ]:
df_ticks['ts'] = df_ticks.ts.astype(str)#.map(lambda x: x[:-3])
updater = ChartUpdater(chart, c.code)
topic = c.code
for idx, row in df_ticks.iloc[10:].iterrows():
    quote = {"Date": row.ts[:10], "Time": row.ts[11:], "Close": [row.close], 'Volume': [row.volume]}
    updater.on_quote(topic, quote)
    time.sleep(0.1)

In [50]:
class ChartUpdater:
    def __init__(self, chart: yvis.PVChart, code: str):
        self.chart = chart
        self.code = code
    
    def on_quote(self, topic, quote):
        code = topic.split('/')[-1]
        if self.code == code:
            dt = f"{quote.get('Date', '').replace('/', '-')} {quote.get('Time', '')[:-3]}"
            price = quote.get('Close', [])
            date = [dt for _ in price]
            volume = quote.get('Volume', [])
            # print(date, price, volume)
            self.chart.update(date, price, volume)

In [30]:
r = df_ticks.iloc[60: 90]
#chart.update(date=r.ts.tolist(), price=r.close.tolist(), volume=r.volume.tolist())

In [45]:
updater = ChartUpdater(chart, c.code)

In [42]:
df_ticks['ts'] = df_ticks.ts.astype(str)#.map(lambda x: x[:-3])

In [46]:
import time

In [26]:
api.quote.set_quote_callback(updater.on_quote)

In [11]:
chart.content.option.tooltip

Etooltip(trigger=<EtooltipTrigger.axis: 'axis'>, axisPointer=EaxisPoint(type=<EaxisPointType.cross: 'cross'>), position=['90%', '10%'])

In [19]:
api.quote.subscribe(api.Contracts.Stocks['2330'])

Response Code: 200 | Event Code: 16 | Info: MKT/*/TSE/2330 | Event: Subscribe or Unsubscribe ok


In [20]:
api.quote.subscribe(api.Contracts.Stocks['2330'], 'bidask')

Response Code: 200 | Event Code: 16 | Info: QUT/*/TSE/2330 | Event: Subscribe or Unsubscribe ok


In [21]:
quote_collector.quotes.keys()

dict_keys(['MKT/idcdmzpcr01/TSE/2330', 'QUT/idcdmzpcr01/TSE/2330'])

In [23]:
ticks = quote_collector.quotes['MKT/idcdmzpcr01/TSE/2330']

In [24]:
[t['Close'] for t in  ticks][:5]

[{'AmountSum': [5945281500.0],
  'Close': [279.5],
  'Date': '2020/03/25',
  'TickType': [1],
  'Time': '09:13:53.755539',
  'VolSum': [21362],
  'Volume': [13]},
 {'AmountSum': [5945560500.0],
  'Close': [279.0],
  'Date': '2020/03/25',
  'TickType': [2],
  'Time': '09:13:54.067044',
  'VolSum': [21363],
  'Volume': [1]},
 {'AmountSum': [5945840000.0],
  'Close': [279.5],
  'Date': '2020/03/25',
  'TickType': [1],
  'Time': '09:13:54.462032',
  'VolSum': [21364],
  'Volume': [1]},
 {'AmountSum': [5946119000.0],
  'Close': [279.0],
  'Date': '2020/03/25',
  'TickType': [2],
  'Time': '09:13:54.911380',
  'VolSum': [21365],
  'Volume': [1]},
 {'AmountSum': [5946398500.0],
  'Close': [279.5],
  'Date': '2020/03/25',
  'TickType': [1],
  'Time': '09:13:55.134611',
  'VolSum': [21366],
  'Volume': [1]}]

In [ ]:
close = [t['Close'][0] for t in  ticks]

In [8]:
fc = sorted([txf for txf in api.Contracts.Futures.TXF], 
            key=lambda x: x.delivery_month)[0]
fc

Future(code='TXFD0', symbol='TXF202004', name='臺股期貨', category='TXF', delivery_month='202004', underlying_kind='I', limit_up=9531.0, limit_down=7799.0, reference=8665.0, update_date='2020/03/24')

In [63]:
datetime.date.today().strftime("%Y-%m-%d")

'2020-03-24'

In [ ]:
ticks_data = api.ticks(fc, date=datetime.date.today().strftime("%Y-%m-%d"), timeout=0)#100000)

In [64]:
ticks_data

Ticks(ts=[], close=[], volume=[], bid_price=[], bid_volume=[], ask_price=[], ask_volume=[])

In [60]:
df_ticks = pd.DataFrame({**ticks_data})

In [61]:
df_ticks

,volume,ask_price,ts,close,bid_price,bid_volume,ask_volume


In [16]:
from collections import defaultdict
class QuoteCollector:
    def __init__(self):
        self.quotes = defaultdict(list)
    
    def append_quote(self, topic: str, quote: dict):
        self.quotes[topic].append(quote)

In [17]:
quote_collector = QuoteCollector()

In [18]:
api.quote.set_quote_callback(quote_collector.append_quote)

In [12]:
api.quote.subscribe(fc)

Response Code: 200 | Event Code: 16 | Info: L/*/TXFD0 | Event: Subscribe or Unsubscribe ok


In [13]:
api.quote.subscribe(fc, 'bidask')

Response Code: 200 | Event Code: 16 | Info: Q/*/TXFD0 | Event: Subscribe or Unsubscribe ok


In [16]:
ticks = quote_collector.quotes[f'L/TFE/{fc.code}']

In [32]:
len(ticks)

573

In [45]:
close = [t['Close'][0] for t in  ticks]

In [46]:
volume = [t['Volume'][0] for t in  ticks]

In [47]:
dt = [t['Date'].replace('/', '-') + ' ' + t['Time'][:-3] for t in  ticks]

In [48]:
st = 0
step = 10
end = st + step
chart = yvis.PVChart(date=dt[st:end], price=close[st:end], volume=volume[st:end])

In [52]:
# chart.content.option.dataset[0].source

In [50]:
for i in range(1, 20):
    st = i * step
    end = st + step
    chart.update(date=dt[st:end], price=close[st:end], volume=volume[st:end])
    #print(i, st, end)